In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv("/kaggle/input/dataset100/TrainingSet_2_Class.csv")
data.head(5)

,label,text
0,1,disabled vehicle westbound highway emily drive...
1,0,new teacher lunch amp training marker wars w s...
2,0,spot uhaultrends canadian destination cites co...
3,0,years ago today mlk gave historic dream speech
4,0,aww itâs hard say goodbye whatâs favorite ...


In [4]:
data["label"].value_counts()

label
1    25549
0    25549
Name: count, dtype: int64

In [ ]:
import spacy
import string
import re

nlp = spacy.load('en_core_web_lg')

stopwords = _stop_words.ENGLISH_STOP_WORDS
lemmatizer = WordNetLemmatizer()

def clean(doc):
    text_no_namedentities = []
    document = nlp(doc)
    ents = [e.text for e in document.ents]
    for item in document:
        if item.text in ents:
            pass
        else:
            text_no_namedentities.append(item.text)
    doc = (" ".join(text_no_namedentities))

    doc = doc.lower().strip()
    doc = doc.replace("</br>", " ") 
    doc = doc.replace("-", " ") 
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([token for token in doc.split() if token not in stopwords])    
    doc = "".join([lemmatizer.lemmatize(word) for word in doc])
    return doc

In [ ]:
data['text'] = data['text'].apply(clean)
data.head()

In [5]:
X = data['text'] 
y = data['label']
print(X.shape, y.shape)

(51098,) (51098,)


In [6]:
from sklearn.model_selection import train_test_split

SEED=123
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(40878,) (40878,)
(10220,) (10220,)


In [7]:
preprocessor = hub.KerasLayer(
    "https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")

encoder = hub.KerasLayer(
    "https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-l-12-h-768-a-12/versions/4",
    trainable=False)

Attaching model 'tensorflow/bert/tensorflow2/en-uncased-preprocess/3' to your Kaggle notebook...
Attaching model 'tensorflow/bert/tensorflow2/en-uncased-l-12-h-768-a-12/4' to your Kaggle notebook...


In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
encoder_inputs = preprocessor(text_input)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]
sequence_output = outputs["sequence_output"]

In [ ]:
# NN
f = tf.keras.layers.Dropout(0.2,name='dropout')(pooled_output)
f=tf.keras.layers.Dense(128,activation='relu',name='hidden')(f)
f=tf.keras.layers.Dense(64,activation='relu',name='hiddenn')(f)
f=tf.keras.layers.Dense(8, activation='relu',name='hiddennn')(f)
f=tf.keras.layers.Dense(1,activation='sigmoid',name='output')(f)

In [ ]:
model=tf.keras.Model(inputs=[text_input],outputs=[f])

In [8]:
X_train = X_train.tolist()
X_test = X_test.tolist()
y_train = y_train.tolist()
y_test = y_test.tolist()

In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train, epochs = 10)

Epoch 1/10
1278/1278 [==============================] - 264s 199ms/step - loss: 0.1979 - accuracy: 0.9248
Epoch 2/10
1278/1278 [==============================] - 254s 199ms/step - loss: 0.1542 - accuracy: 0.9445
Epoch 3/10
1278/1278 [==============================] - 253s 198ms/step - loss: 0.1509 - accuracy: 0.9455
Epoch 4/10
1278/1278 [==============================] - 253s 198ms/step - loss: 0.1478 - accuracy: 0.9461
Epoch 5/10
1278/1278 [==============================] - 253s 198ms/step - loss: 0.1451 - accuracy: 0.9461
Epoch 6/10
1278/1278 [==============================] - 253s 198ms/step - loss: 0.1428 - accuracy: 0.9482
Epoch 7/10
1278/1278 [==============================] - 253s 198ms/step - loss: 0.1394 - accuracy: 0.9499
Epoch 8/10
1278/1278 [==============================] - 254s 199ms/step - loss: 0.1351 - accuracy: 0.9499
Epoch 9/10
1278/1278 [==============================] - 256s 200ms/step - loss: 0.1341 - accuracy: 0.9505
Epoch 10/10
1278/1278 [=======================

In [11]:
model.evaluate(X_test, y_test)

320/320 [==============================] - 64s 197ms/step - loss: 0.1088 - accuracy: 0.9613


[0.10876230895519257, 0.9612524509429932]

In [15]:
y_predict=model.predict(X_test)
y_pred=np.where(y_predict>0.5,1,0) 

320/320 [==============================] - 62s 195ms/step


In [16]:
from sklearn.metrics import classification_report, confusion_matrix

classification_rep = classification_report(y_test, y_pred)

print(classification_rep)

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.98      0.96      5110
           1       0.98      0.95      0.96      5110

    accuracy                           0.96     10220
   macro avg       0.96      0.96      0.96     10220
weighted avg       0.96      0.96      0.96     10220

